# Saving Fetched Data to a CSV

In [ ]:
import requests
import os
import numpy as np
import pandas as pd
from datetime import datetime

# Define the URL and parameters
url = "https://cloudburst-993f5-default-rtdb.firebaseio.com/.json"
params = {
    "auth": "{FIREBASE_API_KEY}"
}

# Fetch data
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    print("Fetched Data:", data)
    
    # Extract relevant information and flatten the structure
    humidity = data['DHT_11']['Humidity']
    temperature = data['DHT_11']['Temperature']
    rain_value = data['Precipitation']['Rain']
    
    # Get the current datetime
    current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Create a DataFrame with the extracted data
    new_data_df = pd.DataFrame({
        'DateTime': [current_datetime],
        'Humidity': [humidity],
        'Temperature': [temperature],
        'Rain': [rain_value], 
    })

    # Define CSV file path
    csv_file = "../artifacts/dataset/fetched_hardware_data.csv"
    
    # Check if the CSV file exists
    if os.path.exists(csv_file):
        # Load existing data
        existing_data_df = pd.read_csv(csv_file)
        
        # Append new data to existing data
        updated_data_df = pd.concat([existing_data_df, new_data_df], ignore_index=True)
    else:
        # If the file doesn't exist, use only the new data
        updated_data_df = new_data_df

    # Save updated data back to CSV
    updated_data_df.to_csv(csv_file, index=False)
    print("Data has been saved and updated in 'fetched_data.csv'")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

Fetched Data: {'DHT_11': {'Humidity': 63, 'Temperature': 27}, 'Precipitation': {'Rain': 4095}}
Data has been saved and updated in 'fetched_data.csv'


## Rain Logic 
Rainfall (mm) = k * (4095 - rain_value)

* rainValue = 3000 -> 5 mm
* rainValue = 1000 -> 20 mm

k = (20 - 5) / (3000m- 1000) = 0.0075 mm per unit

In [2]:
df = pd.read_csv('../artifacts/dataset/fetched_hardware_data.csv')

df['Rain_mm'] = 0.0075 * (4095 - df['Rain'])
df.drop(columns=['Rain'], inplace=True)
df

,Humidity,Temperature,DateTime,Rain_mm
0,73,26.1,NaN,0.0000
1,73,26.2,NaN,3.1800
2,73,26.2,NaN,3.1800
3,73,26.2,NaN,3.1800
4,73,26.2,NaN,2.1900
5,73,26.2,NaN,2.1900
6,73,26.2,NaN,2.1900
7,73,26.2,NaN,2.1900
8,73,26.2,NaN,3.7125
9,73,26.2,NaN,3.7125


## Web-browser Script

ref: https://www.geeksforgeeks.org/schedule-a-python-script-to-run-daily/

In [ ]:
import requests
import pandas as pd
import os
import webbrowser

# Define the URL and parameters
firebase_url = "https://cloudburst-993f5-default-rtdb.firebaseio.com/.json"
params = {
    "auth": "{FIREBASE_API_KEY}"
}

# Fetch data
response = requests.get(firebase_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    print("Fetched Data:", data)
    
    # Convert the data to a DataFrame
    new_data_df = pd.DataFrame([data])  # Adjust based on the structure of `data`

    # Define CSV file path
    csv_file = "fetched_data.csv"
    
    # Check if the CSV file exists
    if os.path.exists(csv_file):
        # Load existing data
        existing_data_df = pd.read_csv(csv_file)
        
        # Append new data to existing data
        updated_data_df = pd.concat([existing_data_df, new_data_df], ignore_index=True)
    else:
        # If the file doesn't exist, use only the new data
        updated_data_df = new_data_df

    # Save updated data back to CSV
    updated_data_df.to_csv(csv_file, index=False)
    print("Data has been saved and updated in 'fetched_data.csv'")

    # Open the CSV file URL in the browser (assumes you upload to Google Drive or a similar service)
    # Here, we're demonstrating opening a link; replace with an actual link if needed
    drive_url = "https://docs.google.com/spreadsheets/u/0/"
    chrome_path = r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe'

    # Register and open in Chrome
    webbrowser.register('chrome', None, webbrowser.BackgroundBrowser(chrome_path))
    webbrowser.get('chrome').open_new_tab(drive_url)
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")